# Str8ts

- Write a number between 1 and N (=size of the grid) into each white cell
- so that no number occurs more than once in a row or column (regardless of the color of the cell).
- The numbers in a horizontal or vertical stripe of consecutive white cells must form a sequence of numbers without gaps, but not necessarily in order (e.g. 2-3-4 or 5-4-7-6).

---------

- 在每个白格中写一个介于 1 和 N 之间的数字（=网格的大小）
- 因此，任何数字都不会在一行或一列中出现超过一次（无论单元格的颜色如何）。
- 连续白格的水平或垂直条纹中的数字必须形成一个没有间隙的数字序列，但不一定按顺序排列（例如 2-3-4 或 5-4-7-6）。

![](https://www.janko.at/Raetsel/Straights/Regeln-01.gif) ![](https://www.janko.at/Raetsel/Straights/Regeln-02.gif)

> Links: https://www.janko.at/Raetsel/Straights/index.htm

In [67]:
def readGrid(path):
    with open(f"../assets/data/Str8t/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("1_6x6")
    print(m, n)
    for g in grid:
        print(g)

6 6
['x', '-', '-', '1x', 'x', 'x']
['x', '-', '-', '-', '5', '-']
['x', '-', '1', '-', '-', '-']
['4', '-', '-', '-', '-', 'x']
['-', '6', '5', '-', '-', 'x']
['x', 'x', 'x', '-', '1', '4x']


In [69]:
from z3 import Solver, Int, Distinct, sat, And, Or, If

def maximum(sol, v, x):
    sol.add(Or([v == x[i] for i in range(len(x))])) # v is an element in x)
    for i in range(len(x)):
        sol.add(v >= x[i]) # and it's the greatest

# v is the minimum value of x
def minimum(sol, v, x):
    sol.add(Or([v == x[i] for i in range(len(x))])) # v is an element in x)
    for i in range(len(x)):
        sol.add(v <= x[i]) # and it's the smallest

def str8t_solver(m, n, grid):
    # every row/column is distinct
    x = [[Int(f"x_{i}_{j}") for j in range(n) ] 
      for i in range(m)]
    s = Solver()
    grid_num = [[0] * n for _ in range(m)]
    
    for i in range(m):
        for j in range(n):
            if grid[i][j].isdigit():
                grid_num[i][j] = int(grid[i][j])
            elif grid[i][j].endswith("x") and len(grid[i][j]) > 1:
                grid_num[i][j] = int(grid[i][j][:-1])
    cells_constr  = [ And(1 <= x[i][j], x[i][j] <= m) 
             for i in range(m) for j in range(n) ]
    # number selection
    
    range_constr = [ If(grid_num[i][j] == 0, 
                  True, 
                  x[i][j] == grid_num[i][j]) 
                    for i in range(m) for j in range(n) ]
    # number constr
    
    rows_constr = []
    # all numbers in a row, unique
    for i in range(m):
        temp = []
        for j in range(n):
            if grid[i][j] == "x":
                continue
            temp.append(x[i][j])
        rows_constr.append(Distinct(temp))
        
    # all numbers in a col, unique
    cols_constr = []
    for j in range(n):
        temp = []
        for i in range(m):
            if grid[i][j] == "x":
                continue
            temp.append(x[i][j])
        cols_constr.append(Distinct(temp))
    
    for i in range(m):
        j = 0
        while j < n:
            while j < n and grid[i][j].endswith("x"):
                j += 1
            temp = []
            while j < n:
                if grid[i][j].endswith("x"):
                    break
                temp.append(x[i][j])
                j += 1
            if len(temp) > 1:
                zMin = Int(f"R_{i}_{j}_zmin")
                zMax = Int(f"R_{i}_{j}_zmax")
                maximum(s, zMax, temp)
                minimum(s, zMin, temp)
                s.add(zMax - zMin == len(temp) - 1 )
    
    for j in range(n):
        i = 0
        while i < m:
            while i < m and grid[i][j].endswith("x"):
                i += 1
            temp = []
            while i < m:
                if grid[i][j].endswith("x"):
                    break
                temp.append(x[i][j])
                i += 1
            if len(temp) > 1:
                zMin = Int(f"C_{i}_{j}_zmin")
                zMax = Int(f"C_{i}_{j}_zmax")
                maximum(s, zMax, temp)
                minimum(s, zMin, temp)
                s.add(zMax - zMin == len(temp) - 1 )

    s.add(rows_constr + cells_constr + cols_constr + range_constr)
    if s.check() == sat:
        model = s.model()
        r = [ [ model.evaluate(x[i][j]) for j in range(n) ] 
              for i in range(m) ]
        for i in range(m):
            for j in range(n):
                if grid[i][j] == "x":
                    print("x", end = " ")
                else:
                    print(r[i][j], end = " ")
            print("")
    else:
        print ("failed to solve")


if __name__ == "__main__":
    m, n, grid = readGrid("1_6x6")
    str8t_solver(m, n, grid)

x 4 3 1 x x 
x 2 4 3 5 1 
x 3 1 5 4 2 
4 5 2 6 3 x 
3 6 5 4 2 x 
x x x 2 1 4 
